## Pipeline for microendoscopic data processing in CaImAn using the CNMF-E algorithm
This demo presents a complete pipeline for processing microendoscopic data using CaImAn. It includes:
- Motion Correction using the NoRMCorre algorithm
- Source extraction using the CNMF-E algorithm
- Deconvolution using the OASIS algorithm

Some basic visualization is also included. The demo illustrates how to `params`, `MoctionCorrection` and `cnmf` object for processing 1p microendoscopic data. For processing two-photon data consult the related `demo_pipeline.ipynb` demo. For more information see the companion CaImAn paper.

In [ ]:
try:
    get_ipython().run_line_magic('load_ext', 'autoreload')
    get_ipython().run_line_magic('autoreload', '2')
    get_ipython().run_line_magic('matplotlib', 'qt')
except:
    pass

import logging
import matplotlib.pyplot as plt
import numpy as np

logging.basicConfig(format=
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
                    # filename="caiman.log",
                    level=logging.DEBUG)

import caiman as cm
from caiman.source_extraction import cnmf
from caiman.utils.utils import download_demo
from caiman.utils.visualization import inspect_correlation_pnr, nb_inspect_correlation_pnr
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import params as params
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
from caiman.source_extraction.cnmf.initialization import init_neurons_corr_pnr
from caiman.utils.visualization import view_quilt
import cv2
import pickle
import scipy.io as sio

try:
    cv2.setNumThreads(0)
except:
    pass
import bokeh.plotting as bpl
import holoviews as hv
bpl.output_notebook()
hv.notebook_extension('bokeh')

#%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)
%autoreload 2

### Select file(s) to be processed
The `download_demo` function will download the specific file for you and return the complete path to the file which will be stored in your `caiman_data` directory. If you adapt this demo for your data make sure to pass the complete path to your file(s). Remember to pass the `fnames` variable as a list. Note that the memory requirement of the CNMF-E algorithm are much higher compared to the standard CNMF algorithm. Test the limits of your system before trying to process very large amounts of data.

In [ ]:
fnames = [r'Y:\CM2scope\trace_fear_conditioning\train\gzc_rasgrf-ai148d-93\My_V4_Miniscope\AMF_despeckle_MC_denoised_8bit.tif'.replace('\\','\\'),
         ]
print(fnames)

In [ ]:
for fname in fnames:
    name = fname.split(".")[0]+'_variables.pkl'
    with open(name, 'rb') as file:
        vars = pickle.load(file)
        cn_filter = vars['cn_filter']
        pnr = vars['pnr']
        data_max = vars['data_max']
        data_noise = vars['data_noise']
        std = vars['std']
        opts = vars['opts']
        
    cnm = cnmf.CNMF(n_processes=n_processes, dview=dview, Ain=None, params=opts)
    
    name = fname.split(".")[0]+'_cnm_estimates.pkl'
    with open(name, 'rb') as file:
        cnm.estimates = pickle.load(file)
    
    name = fname.split(".")[0]+'_correlation_images_sigma2.png'
    pnr = cv2.imread(name, cv2.IMREAD_ANYDEPTH).astype(np.float32)
    
    name = fname.split(".")[0]+'pnr_images_sigma2.png'
    cn_filter = cv2.imread(name, cv2.IMREAD_ANYDEPTH).astype(np.float32)
    
    percentile_value = np.percentile(cn_filter[cn_filter > 0], 99.9)
    mask = (cn_filter > percentile_value)
    cn_filter[mask] = percentile_value
    
    
    name=fname.split(".")[0]+'_caiman_result.mat'
    mat = sio.loadmat(name)
    cnm.estimates.idx_components=np.squeeze(mat['A_neuron_good_idx'].astype(np.int32))
    cnm.estimates.idx_components_bad=np.squeeze(mat['A_neuron_bad_idx'].astype(np.int32))
    
    #%% plot contour plots of accepted and rejected components
    cnm.estimates.plot_contours_nb(img=pnr, idx=cnm.estimates.idx_components,thr=0.8,cmap='Greys',thr_method='nrg')
    del cnm